In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1612273567887_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, r.review_length, r.review_rating, r.review_would_recommend, \
case when (contains(r.observations, 'CC') )  then 1 else 0 end as CC \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US','en_CA','en_GB') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and r.created_date > '2020-01-01' \
"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head()
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID       NAME  ... REVIEW_WOULD_RECOMMEND CC
0  311131522        UGG  ...                  False  0
1  311145813  Dillard's  ...                   True  0
2  311254424  Dillard's  ...                   True  0
3  310819090   Nautilus  ...                   None  0
4  310743609        UGG  ...                   True  0

[5 rows x 8 columns]

In [6]:
reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + '. '+reviews['REVIEW_COMMENTS']
#Use set_option so text doesn't truncate

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#reviews.head()

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8086374 entries, 0 to 8086373
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   UGC_ID                  int64  
 1   NAME                    object 
 2   REVIEW_HEADLINE         object 
 3   REVIEW_COMMENTS         object 
 4   REVIEW_LENGTH           float64
 5   REVIEW_RATING           int64  
 6   REVIEW_WOULD_RECOMMEND  object 
 7   CC                      int64  
 8   ALL_TEXT                object 
dtypes: float64(1), int64(3), object(5)
memory usage: 555.2+ MB

In [8]:
#Remove nulls so sentiment analyzer works
reviews=reviews[reviews['ALL_TEXT'].notnull()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Run sentiment on headline and comments
#Compare to star rating
#Compare to bottom line/would recommend Y/N

#No sentiment unless recommend option is null
sentiment_analyzer = SentimentIntensityAnalyzer()
reviews['sentiment_score'] = np.nan
#reviews['sentiment_score'] = reviews['ALL_TEXT'].apply(lambda x: sentiment_analyzer.polarity_scores(x)['compound'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [11]:
reviews_recommend = reviews[reviews['REVIEW_WOULD_RECOMMEND'].notnull()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
reviews_recommend.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4062637 entries, 0 to 8086373
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   UGC_ID                  int64  
 1   NAME                    object 
 2   REVIEW_HEADLINE         object 
 3   REVIEW_COMMENTS         object 
 4   REVIEW_LENGTH           float64
 5   REVIEW_RATING           int64  
 6   REVIEW_WOULD_RECOMMEND  object 
 7   CC                      int64  
 8   ALL_TEXT                object 
 9   sentiment_score         float64
dtypes: float64(2), int64(3), object(5)
memory usage: 341.0+ MB

In [13]:
reviews_recommend['model_CC'] = 0

#reviews_recommend.loc[( ((reviews_recommend['REVIEW_RATING']==5) | (reviews_recommend['REVIEW_RATING']==4)) & (reviews_recommend['REVIEW_WOULD_RECOMMEND']==False)  ) |
#            ( ((reviews_recommend['REVIEW_RATING']==1) | (reviews_recommend['REVIEW_RATING']==2)) & (reviews_recommend['REVIEW_WOULD_RECOMMEND']==True) ) , 'model_CC'] = 1

reviews_recommend.loc[( ((reviews_recommend['REVIEW_RATING']==5) ) & (reviews_recommend['REVIEW_WOULD_RECOMMEND']==False)  ) |
            ( ((reviews_recommend['REVIEW_RATING']==1)) & (reviews_recommend['REVIEW_WOULD_RECOMMEND']==True) ) , 'model_CC'] = 1


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib64/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)

In [14]:
#reviews_recommend.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
reviews_nullrecommend = reviews[reviews['REVIEW_WOULD_RECOMMEND'].isnull()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
reviews_nullrecommend.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4022781 entries, 3 to 8086372
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   UGC_ID                  int64  
 1   NAME                    object 
 2   REVIEW_HEADLINE         object 
 3   REVIEW_COMMENTS         object 
 4   REVIEW_LENGTH           float64
 5   REVIEW_RATING           int64  
 6   REVIEW_WOULD_RECOMMEND  object 
 7   CC                      int64  
 8   ALL_TEXT                object 
 9   sentiment_score         float64
dtypes: float64(2), int64(3), object(5)
memory usage: 337.6+ MB

In [17]:
reviews_nullrecommend['sentiment_score'] = reviews_nullrecommend['ALL_TEXT'].apply(lambda x: sentiment_analyzer.polarity_scores(x)['compound'])
reviews_nullrecommend.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID       NAME  \
3   310819090  Nautilus    
13  310820187  Dillard's   
14  310793588  Dillard's   
16  311005702  Crocs US    
17  311098397  Crocs US    

                                              REVIEW_HEADLINE  \
3   Terrible customer service, my order got messed up 3 times   
13  Yes!!!!                                                     
14  Yes                                                         
16  would tell anyone to buy it                                 
17  Comfortable and fast shipping.                              

                                                                                     REVIEW_COMMENTS  \
3   biking                                                                                             
13  I'm happy with my purchase! Comfortable espadrilles. And they look great with jeans of any kind.   
14  Love                                                                                               
16  Loved it. Very comfo

In [18]:
reviews_nullrecommend['model_CC'] = 0

#Char limit needed bc otherwise too many FP where sentiment seems highly positive but not or other way around
#reviews_nullrecommend.loc[(((reviews_nullrecommend['REVIEW_RATING']==5) | (reviews_nullrecommend['REVIEW_RATING']==4)) 
#                           & (reviews_nullrecommend['sentiment_score']<-0.75) & (reviews_nullrecommend['REVIEW_LENGTH']<20) )  |
#        (((reviews_nullrecommend['REVIEW_RATING']==1) | (reviews_nullrecommend['REVIEW_RATING']==2)) 
#        & (reviews_nullrecommend['sentiment_score']>0.75) & (reviews_nullrecommend['REVIEW_LENGTH']<20) )  , 'model_CC'] = 1

reviews_nullrecommend.loc[(((reviews_nullrecommend['REVIEW_RATING']==5) ) 
                           & (reviews_nullrecommend['sentiment_score']<-0.75) & (reviews_nullrecommend['REVIEW_LENGTH']<20) )  |
        (((reviews_nullrecommend['REVIEW_RATING']==1) ) 
        & (reviews_nullrecommend['sentiment_score']>0.75) & (reviews_nullrecommend['REVIEW_LENGTH']<20) )  , 'model_CC'] = 1


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib64/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)

In [19]:
#reviews_nullrecommend[reviews_nullrecommend['model_CC']==1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
reviews = reviews_recommend.append(reviews_nullrecommend)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID       NAME                                     REVIEW_HEADLINE  \
0  311131522  UGG        Noisy When Walking                                   
1  311145813  Dillard's  Great light purple🙂                                  
2  311254424  Dillard's  Not for that price                                   
4  310743609  UGG        super cute                                           
5  311045216  UGG        I would buy this item again if it was in a 2X size   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [22]:
FP = reviews[(reviews['CC'] == 0) & (reviews['model_CC'] == 1)]
FN = reviews[(reviews['CC'] == 1) & (reviews['model_CC'] == 0)]
TP = reviews[(reviews['CC'] == 1) & (reviews['model_CC'] == 1)]
TN = reviews[(reviews['CC'] == 0) & (reviews['model_CC'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

32050

In [29]:
count = FP[FP['sentiment_score'].isna()]
count.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UGC_ID                    31821
NAME                      31821
REVIEW_HEADLINE           31821
REVIEW_COMMENTS           31821
REVIEW_LENGTH             31821
REVIEW_RATING             31821
REVIEW_WOULD_RECOMMEND    31821
CC                        31821
ALL_TEXT                  31821
sentiment_score           0    
model_CC                  31821
dtype: int64

In [24]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

28640

In [25]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4686

In [26]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8020042

In [27]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("REVIEW_LENGTH", FloatType(), True)\
,StructField("REVIEW_RATING", StringType(), True)\
,StructField("REVIEW_WOULD_RECOMMEND", BooleanType(), True)\
,StructField("CC", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("sentiment_score", FloatType(), True)\
,StructField("model_CC", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP= spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CC_FP_test4_only1or5star").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.CC_FN_TEST7")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.CC_FP_TEST7")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…